In [ ]:
import cv2
import numpy as np
import networkx as nx
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Input
from keras.utils import to_categorical
from skimage.feature import local_binary_pattern

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"

# Helper function to load images
def load_images(path):
    images = []
    labels = []
    for label in ['Positive', 'Negative']:
        for file in glob.glob(os.path.join(path, label, '*.jpg')):
            img = cv2.imread(file)
            if img is not None:
                img = cv2.resize(img, (128, 128))
                images.append(img)
                labels.append(1 if label == 'Positive' else 0)
    return np.array(images), np.array(labels)

# Load datasets
train_images, train_labels = load_images(train_path)
test_images, test_labels = load_images(test_path)
val_images, val_labels = load_images(val_path)

# Function to build the graph representation of the image
def build_graph(image):
    graph = nx.Graph()
    h, w, _ = image.shape
    for i in range(h):
        for j in range(w):
            graph.add_node((i, j), pixel=image[i, j])
            if i > 0:
                graph.add_edge((i, j), (i-1, j), weight=np.linalg.norm(image[i, j] - image[i-1, j]))
            if j > 0:
                graph.add_edge((i, j), (i, j-1), weight=np.linalg.norm(image[i, j] - image[i, j-1]))
    return graph

# Extract features from the graph (example using node degrees)
def extract_graph_features(graph):
    degrees = [d for n, d in graph.degree()]
    return np.array(degrees)

# Extract LBP features from the image
def extract_image_features(image):
    lbp = local_binary_pattern(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

# Build and compile a CNN model for image feature extraction
def build_cnn_model(input_shape):
    cnn_input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(cnn_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    cnn_model = Model(inputs=cnn_input, outputs=x)
    return cnn_model

# Encode labels
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)
val_labels = le.transform(val_labels)

# Convert labels to categorical (one-hot encoding)
train_labels = to_categorical(train_labels, num_classes=2)
test_labels = to_categorical(test_labels, num_classes=2)
val_labels = to_categorical(val_labels, num_classes=2)

# Create graph representations of images
train_graphs = [build_graph(img) for img in train_images]
test_graphs = [build_graph(img) for img in test_images]
val_graphs = [build_graph(img) for img in val_images]

# Extract features from graphs
train_graph_features = np.array([extract_graph_features(graph) for graph in train_graphs])
test_graph_features = np.array([extract_graph_features(graph) for graph in test_graphs])
val_graph_features = np.array([extract_graph_features(graph) for graph in val_graphs])

# Extract features from images
train_image_features = np.array([extract_image_features(img) for img in train_images])
test_image_features = np.array([extract_image_features(img) for img in test_images])
val_image_features = np.array([extract_image_features(img) for img in val_images])

# Reshape image data for CNN
train_images_cnn = np.reshape(train_images, (train_images.shape[0], 128, 128, 3))
test_images_cnn = np.reshape(test_images, (test_images.shape[0], 128, 128, 3))
val_images_cnn = np.reshape(val_images, (val_images.shape[0], 128, 128, 3))

input_shape = (128, 128, 3)

# Build CNN model for image feature extraction
cnn_model = build_cnn_model(input_shape)

# Define the full model combining CNN and graph features
graph_input = Input(shape=(train_graph_features.shape[1],))
cnn_features = cnn_model.output
combined = Concatenate()([cnn_features, graph_input])
x = Dense(64, activation='relu')(combined)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=[cnn_model.input, graph_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([train_images_cnn, train_graph_features], train_labels, epochs=10, batch_size=32, validation_data=([val_images_cnn, val_graph_features], val_labels))

# Evaluate the model
y_pred = model.predict([test_images_cnn, test_graph_features])
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(test_labels, axis=1)
print("Combined Model Classification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=['Negative', 'Positive']))


In [1]:
import cv2
import numpy as np
import networkx as nx
import os
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Input
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from skimage.feature import local_binary_pattern

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"

# Function to build the graph representation of the image
def build_graph(image):
    graph = nx.Graph()
    h, w, _ = image.shape
    for i in range(h):
        for j in range(w):
            graph.add_node((i, j), pixel=image[i, j])
            if i > 0:
                graph.add_edge((i, j), (i-1, j), weight=np.linalg.norm(image[i, j] - image[i-1, j]))
            if j > 0:
                graph.add_edge((i, j), (i, j-1), weight=np.linalg.norm(image[i, j] - image[i, j-1]))
    return graph

# Extract features from the graph (example using node degrees)
def extract_graph_features(graph):
    degrees = [d for n, d in graph.degree()]
    return np.array(degrees)

# Extract LBP features from the image
def extract_image_features(image):
    lbp = local_binary_pattern(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

# Generator to yield batches of images and labels
def image_generator(path, batch_size=32):
    while True:
        images = []
        graph_features = []
        labels = []
        for label in ['Positive', 'Negative']:
            for file in glob.glob(os.path.join(path, label, '*.jpg')):
                img = cv2.imread(file)
                if img is not None:
                    img = cv2.resize(img, (128, 128))
                    graph = build_graph(img)
                    graph_feat = extract_graph_features(graph)
                    image_feat = extract_image_features(img)

                    images.append(img)
                    graph_features.append(graph_feat)
                    labels.append(1 if label == 'Positive' else 0)

                    if len(images) == batch_size:
                        yield [np.array(images), np.array(graph_features)], to_categorical(np.array(labels), num_classes=2)
                        images = []
                        graph_features = []
                        labels = []

        if images:
            yield [np.array(images), np.array(graph_features)], to_categorical(np.array(labels), num_classes=2)

# Build and compile a CNN model for image feature extraction
def build_cnn_model(input_shape):
    cnn_input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(cnn_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    cnn_model = Model(inputs=cnn_input, outputs=x)
    return cnn_model

input_shape = (128, 128, 3)

# Build CNN model for image feature extraction
cnn_model = build_cnn_model(input_shape)

# Define the full model combining CNN and graph features
graph_input = Input(shape=(128*128,))
cnn_features = cnn_model.output
combined = Concatenate()([cnn_features, graph_input])
x = Dense(64, activation='relu')(combined)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=[cnn_model.input, graph_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define batch size and number of steps
batch_size = 32
train_steps = sum([len(files) for r, d, files in os.walk(train_path)]) // batch_size
val_steps = sum([len(files) for r, d, files in os.walk(val_path)]) // batch_size
test_steps = sum([len(files) for r, d, files in os.walk(test_path)]) // batch_size

# Train the model
model.fit(image_generator(train_path, batch_size), steps_per_epoch=train_steps, epochs=10, validation_data=image_generator(val_path, batch_size), validation_steps=val_steps)

# Evaluate the model
y_pred = model.predict(image_generator(test_path, batch_size), steps=test_steps)
y_pred_classes = np.argmax(y_pred, axis=1)

# Since test labels are not available in generator, manually load test labels
test_labels = []
for label in ['Positive', 'Negative']:
    for file in glob.glob(os.path.join(test_path, label, '*.jpg')):
        test_labels.append(1 if label == 'Positive' else 0)

print("Combined Model Classification Report:")
print(classification_report(test_labels, y_pred_classes, target_names=['Negative', 'Positive']))


Epoch 1/10
126/126 [==============================] - 559s 4s/step - loss: 942.0775 - accuracy: 0.9640 - val_loss: 5140.6475 - val_accuracy: 0.8042
Epoch 2/10
 55/126 [============>.................] - ETA: 4:26 - loss: 2024.7341 - accuracy: 0.5017

KeyboardInterrupt: 

In [6]:
import cv2
import numpy as np
import os
import glob
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Concatenate
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from skimage.feature import local_binary_pattern
from sklearn.metrics import classification_report

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"

# Function to extract LBP features from the image
def extract_lbp_features(image):
    lbp = local_binary_pattern(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

# Generator to yield batches of images, LBP features, and labels
def image_lbp_generator(path, batch_size=32):
    while True:
        images = []
        lbp_features = []
        labels = []
        for label in ['Positive', 'Negative']:
            for file in glob.glob(os.path.join(path, label, '*.jpg')):
                img = cv2.imread(file)
                if img is not None:
                    img = cv2.resize(img, (128, 128))
                    lbp_feat = extract_lbp_features(img)
                    
                    images.append(img)
                    lbp_features.append(lbp_feat)
                    labels.append(1 if label == 'Positive' else 0)

                    if len(images) == batch_size:
                        yield [np.array(images), np.array(lbp_features)], to_categorical(np.array(labels), num_classes=2)
                        images = []
                        lbp_features = []
                        labels = []

        if images:
            yield [np.array(images), np.array(lbp_features)], to_categorical(np.array(labels), num_classes=2)

# Build and compile a CNN model for image feature extraction
def build_cnn_model(input_shape):
    cnn_input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(cnn_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    cnn_model = Model(inputs=cnn_input, outputs=x)
    return cnn_model

input_shape = (128, 128, 3)

# Build CNN model for image feature extraction
cnn_model = build_cnn_model(input_shape)

# Define the full model combining CNN and LBP features
lbp_input = Input(shape=(26,))
cnn_features = cnn_model.output
combined = Concatenate()([cnn_features, lbp_input])
x = Dense(64, activation='relu')(combined)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=[cnn_model.input, lbp_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define batch size and number of steps
batch_size = 32
train_steps = sum([len(files) for r, d, files in os.walk(train_path)]) // batch_size
val_steps = sum([len(files) for r, d, files in os.walk(val_path)]) // batch_size
test_steps = sum([len(files) for r, d, files in os.walk(test_path)]) // batch_size

# Train the model
model.fit(image_lbp_generator(train_path, batch_size), steps_per_epoch=train_steps, epochs=10, validation_data=image_lbp_generator(val_path, batch_size), validation_steps=val_steps)

# Manually load test data
test_images = []
test_lbp_features = []
test_labels = []
for label in ['Positive', 'Negative']:
    for file in glob.glob(os.path.join(test_path, label, '*.jpg')):
        img = cv2.imread(file)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            lbp_feat = extract_lbp_features(img)
            test_images.append(img)
            test_lbp_features.append(lbp_feat)
            test_labels.append(1 if label == 'Positive' else 0)

# Convert to numpy arrays
test_images = np.array(test_images)
test_lbp_features = np.array(test_lbp_features)
test_labels = np.array(test_labels)

# Predict on the entire test dataset
y_pred = model.predict([test_images, test_lbp_features])
y_pred_classes = np.argmax(y_pred, axis=1)

# Print classification report
print("Combined Model Classification Report:")
print(classification_report(test_labels, y_pred_classes, target_names=['Negative', 'Positive']))





Epoch 1/10
126/126 [==============================] - 8s 61ms/step - loss: 0.4490 - accuracy: 0.9960 - val_loss: 12555.9277 - val_accuracy: 0.8042
Epoch 2/10
126/126 [==============================] - 7s 55ms/step - loss: 1233.1373 - accuracy: 0.8754 - val_loss: 948.1520 - val_accuracy: 0.8042
Epoch 3/10
126/126 [==============================] - 7s 54ms/step - loss: 94.5656 - accuracy: 0.8022 - val_loss: 5.5714 - val_accuracy: 0.8062
Epoch 4/10
126/126 [==============================] - 7s 55ms/step - loss: 2.6123 - accuracy: 0.7509 - val_loss: 3.9454 - val_accuracy: 0.7979
Epoch 5/10
126/126 [==============================] - 7s 54ms/step - loss: 1.2821 - accuracy: 0.8064 - val_loss: 3.1915 - val_accuracy: 0.7958
Epoch 6/10
126/126 [==============================] - 7s 55ms/step - loss: 0.8802 - accuracy: 0.8600 - val_loss: 2.8048 - val_accuracy: 0.8104
Epoch 7/10
126/126 [==============================] - 7s 55ms/step - loss: 0.5620 - accuracy: 0.9001 - val_loss: 2.3449 - val_accura

In [ ]:
import cv2
import numpy as np
import os
import glob
import networkx as nx
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Concatenate, Dropout
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from skimage.feature import local_binary_pattern
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"

# Function to extract LBP features from the image
def extract_lbp_features(image):
    lbp = local_binary_pattern(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

# Function to build graph and extract weight features
def build_graph(image):
    graph = nx.Graph()
    h, w, _ = image.shape
    for i in range(h):
        for j in range(w):
            graph.add_node((i, j), pixel=image[i, j])
            if i > 0:
                graph.add_edge((i, j), (i-1, j), weight=np.linalg.norm(image[i, j] - image[i-1, j]))
            if j > 0:
                graph.add_edge((i, j), (i, j-1), weight=np.linalg.norm(image[i, j] - image[i, j-1]))
    return graph

def extract_graph_weight_features(image):
    graph = build_graph(image)
    weights = [graph[u][v]['weight'] for u, v in graph.edges()]
    weights_hist, _ = np.histogram(weights, bins=30)
    weights_hist = weights_hist.astype("float")
    weights_hist /= (weights_hist.sum() + 1e-6)
    return weights_hist

# Generator to yield batches of images, LBP features, graph weight features, and labels
def image_lbp_graph_generator(path, batch_size=32, augment=False):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    ) if augment else None
    
    while True:
        images = []
        lbp_features = []
        graph_weight_features = []
        labels = []
        for label in ['Positive', 'Negative']:
            for file in glob.glob(os.path.join(path, label, '*.jpg')):
                img = cv2.imread(file)
                if img is not None:
                    img = cv2.resize(img, (128, 128))
                    lbp_feat = extract_lbp_features(img)
                    graph_weight_feat = extract_graph_weight_features(img)
                    
                    if augment:
                        img = img.reshape((1,) + img.shape)
                        aug_iter = datagen.flow(img)
                        img = next(aug_iter)[0]
                    
                    images.append(img)
                    lbp_features.append(lbp_feat)
                    graph_weight_features.append(graph_weight_feat)
                    labels.append(1 if label == 'Positive' else 0)

                    if len(images) == batch_size:
                        yield [np.array(images), np.array(lbp_features), np.array(graph_weight_features)], to_categorical(np.array(labels), num_classes=2)
                        images = []
                        lbp_features = []
                        graph_weight_features = []
                        labels = []

        if images:
            yield [np.array(images), np.array(lbp_features), np.array(graph_weight_features)], to_categorical(np.array(labels), num_classes=2)

# Build and compile a CNN model for image feature extraction
def build_cnn_model(input_shape):
    cnn_input = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(cnn_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    cnn_model = Model(inputs=cnn_input, outputs=x)
    return cnn_model

input_shape = (128, 128, 3)

# Build CNN model for image feature extraction
cnn_model = build_cnn_model(input_shape)

# Define the full model combining CNN, LBP, and Graph Weight features
lbp_input = Input(shape=(26,))
graph_weight_input = Input(shape=(30,))
cnn_features = cnn_model.output
combined = Concatenate()([cnn_features, lbp_input, graph_weight_input])
x = Dense(128, activation='relu')(combined)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=[cnn_model.input, lbp_input, graph_weight_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define batch size and number of steps
batch_size = 32
train_steps = sum([len(files) for r, d, files in os.walk(train_path)]) // batch_size
val_steps = sum([len(files) for r, d, files in os.walk(val_path)]) // batch_size
test_steps = sum([len(files) for r, d, files in os.walk(test_path)]) // batch_size

# Calculate class weights
train_labels = []
for label in ['Positive', 'Negative']:
    for file in glob.glob(os.path.join(train_path, label, '*.jpg')):
        train_labels.append(1 if label == 'Positive' else 0)
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = dict(enumerate(class_weights))

# Early stopping and model checkpoint
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Train the model with data augmentation
model.fit(
    image_lbp_graph_generator(train_path, batch_size, augment=True),
    steps_per_epoch=train_steps,
    epochs=10,
    validation_data=image_lbp_graph_generator(val_path, batch_size),
    validation_steps=val_steps,
    class_weight=class_weights,
    callbacks=callbacks
)

# Manually load test data
test_images = []
test_lbp_features = []
test_graph_weight_features = []
test_labels = []

for label in ['Positive', 'Negative']:
    for file in glob.glob(os.path.join(test_path, label, '*.jpg')):
        img = cv2.imread(file)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            lbp_feat = extract_lbp_features(img)
            graph_weight_feat = extract_graph_weight_features(img)
            test_images.append(img)
            test_lbp_features.append(lbp_feat)
            test_graph_weight_features.append(graph_weight_feat)
            test_labels.append(1 if label == 'Positive' else 0)

test_images = np.array(test_images)
test_lbp_features = np.array(test_lbp_features)
test_graph_weight_features = np.array(test_graph_weight_features)
test_labels = np.array(test_labels)

# Make predictions on the test data
y_pred = model.predict([test_images, test_lbp_features, test_graph_weight_features])
y_pred_classes = np.argmax(y_pred, axis=1)

# Print the classification report
print("Combined Model Classification Report:")
print(classification_report(test_labels, y_pred_classes, target_names=['Negative', 'Positive']))


Epoch 1/10
126/126 [==============================] - 705s 6s/step - loss: 28.9956 - accuracy: 0.9541 - val_loss: 351.8496 - val_accuracy: 0.8042
Epoch 2/10
  1/126 [..............................] - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.0000

/Users/niteshyadav/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


126/126 [==============================] - 686s 5s/step - loss: 84.0517 - accuracy: 0.7030 - val_loss: 372.1590 - val_accuracy: 0.8042
Epoch 3/10
126/126 [==============================] - 674s 5s/step - loss: 60.9628 - accuracy: 0.8184 - val_loss: 356.1048 - val_accuracy: 0.8042
Epoch 4/10
126/126 [==============================] - 680s 5s/step - loss: 60.0018 - accuracy: 0.5264 - val_loss: 4.9255 - val_accuracy: 0.8042
Epoch 5/10
126/126 [==============================] - 669s 5s/step - loss: 5.6073 - accuracy: 0.3214 - val_loss: 0.6736 - val_accuracy: 0.7667
Epoch 6/10
 28/126 [=====>........................] - ETA: 4:18 - loss: 1.6368 - accuracy: 0.4636